# Dappradar

Dappradar (https://dappradar.com/) aggregates dapp data across multiple chains. They recently launched an API service at https://api-portal.dappradar.com/home.

The API docs are at: https://api-docs.dappradar.com/

# Get list of Dapps on Cronos chain

In [ ]:
# To install dependencies: pip install -r requirements.txt
import os
import requests
import json
import time
from dotenv import load_dotenv
load_dotenv(override=True)

dappradar_url = os.environ.get('DAPPRADAR_URL')
dappradar_api_key = os.environ.get('DAPPRADAR_API_KEY')

headers = {
    'X-BLOBR-KEY': dappradar_api_key,
}

def get_list_of_dapps():
    current_page = 1
    number_pages = 1
    res = []
    while current_page <= number_pages:
        api_response = requests.request(
            method="get",
            url=f"{dappradar_url}/dapps",
            headers=headers,
            params={
                'chain': 'cronos',
                'page': current_page,
            }
        )
        if api_response.status_code != 200:
            raise Exception(
                "Request returned an error: {} {}".format(
                    api_response.status_code, api_response.text
                )
            )
        res = res + api_response.json()['results']
        number_pages = api_response.json()['pageCount']
        print("Downloaded page {} of {}...".format(current_page, number_pages))
        current_page += 1
        if current_page <= number_pages:
            time.sleep(2)
    return res

list_of_dapps = get_list_of_dapps()
print("\nDownloaded {} dapps".format(len(list_of_dapps)))
print("\nExample dapp:")
print(json.dumps(list_of_dapps[0], indent=4, sort_keys=True))
with open('./outputs/dappradar_output.json', 'w') as f:
    f.write(json.dumps(list_of_dapps, indent=4, sort_keys=True))


# Find dapp ID based on partial text string

In [ ]:
def find_dappid(dapp_name: str, list_of_dapps):
    res = []
    for item in list_of_dapps:
        if dapp_name.lower() in item['name'].lower():
            res.append([item['name'], item['dappId']])
    return res

print("Searching for VVS dapp ID...")
print(json.dumps(find_dappid('VVS', list_of_dapps), indent=4, sort_keys=True))

# Get dapp data

This endpoint returns data (users, transactions, volume, balance) from a single dapp that can be seen on the single dapp page. You need to input the dappId, as well as the timeframe and chains the data should be retrieved from. The dappId can be obtained by going through the paginated list, or more convenient for a single dapp by taking it from the dapp's about page (Example: https://dappradar.com/cronos/defi/vvs-finance/about).

In [ ]:
def get_dapp_data(dapp_id: int):
    api_response = requests.request(
        method="get",
        url=f"{dappradar_url}/dapps/{dapp_id}",
        headers=headers,
        params={
            'chain': 'cronos',
            'range': '30d' # Can be 24h, 7d or 30d
        }
    )
    if api_response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                api_response.status_code, api_response.text
            )
        )
    return api_response.json()['results']

print("Example dapp data:")
dapp_data = get_dapp_data(13220)
print(json.dumps(dapp_data, indent=4, sort_keys=True))